In [2]:
# Some standard import statements that are nice for Jupyter

import sys
import os
import logging

import numpy as np

from scipy.optimize import curve_fit

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.dpi'] = 200

import time

import json

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

# importing my model functions for analysis
from psi_transmission.model import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Organization of the data files

In a new directory, at data_main/sorted, I have copied only the good run files (with this determination precipitating from the work in the parsing_elog notebooks). I have renamed every .tof and every .txt file according to the following convention: inheritedFileName_configuration_runtype.tof(txt). Here are my conventions for configuration and runtype:

- NORM - normalization
- JPTI - JP Ti guide with NiP
- JPSU - JP SUS guide with NiP
- DISK - SS Disk
- GD01 - UGD01 guide
- GD03 - UGD03 guide
- EPSU - EP SUS guide with NiP


- s005 - 5 second storage measurement
- s020 - 20 second storage measurement
- s100 - 100 second storage measurement
- shot - direct shot measurement

For example:

T101217_0222_EPSU_s100.tof

T091217_0130_NORM_s100.txt

### Master Run List:

Run #'s                      | Day |  Storage Time (s)   | Configuration          |
---                          | --- |      ---            | ---                    |
--- - 21                     | 8   |      n/a            | BAD                    |
 22 - 24                     | 8   |       20            | normalization          |
 25 - 31                     | 8   |      100            | normalization          |
 32 - 35*                    | 8   |        5            | normalization          |
      48                     | 8   |   direct            | JP SUS guide with NiP  |
      49                     | 8   |   direct            | BAD                    |
 50 - 60                     | 8   |      100            | JP SUS guide with NiP  |
 61 - 63                     | 8   |        5            | JP SUS guide with NiP  |
 64 - 66*                    | 8   |       20            | JP SUS guide with NiP  |
      76                     | 8   |   direct            | normalization          |
 77 - 79                     | 8   |        5            | normalization          |
 80 - 82                     | 8   |       20            | normalization          |
 83 - 88$^\dagger$           | 8   |      100            | normalization          |
      98                     | 8   |   direct            | JP Ti guide with NiP   |
 99 - 105                    | 8   |      100            | JP Ti guide with NiP   |
  1 - 90                     | 9   |      100            | JP Ti guide with NiP   |
 99 - 101                    | 9   |       20            | JP Ti guide with NiP   |
102 - 110                    | 9   |        5            | JP Ti guide with NiP   |
      120                    | 9   |   direct            | normalization          |
121 - 123                    | 9   |        5            | normalization          |
124 - 125                    | 9   |       20            | normalization          |
      126                    | 9   |       20            | BAD                    |
      127                    | 9   |       20            | normalization          |
128 - 135                    | 9   |      100            | normalization          |
      145                    | 9   |   direct            | SS disk                |
146 - 153                    | 9   |      100            | SS disk                |
154 - 156                    | 9   |       20            | SS disk                |
157 - 159                    | 9   |        5            | SS disk                |
      175                    | 9   |   direct            | normalization          |
176 - 183                    | 9   |      100            | normalization          |
184 - 186                    | 9   |       20            | normalization          |
187 - 189                    | 9   |        5            | normalization          |
      198                    | 9   |   direct            | UGD01                  |
      199                    | 9   |      100            | UGD01                  |
  1 - 107                    | 10  |      100            | UGD01                  |
108 - 110                    | 10  |       20            | UGD01                  |
111 - 113                    | 10  |        5            | UGD01                  |
      122                    | 10  |   direct            | normalization          |
123 - 130                    | 10  |      100            | normalization          |
131 - 133                    | 10  |       20            | normalization          |
134 - 136                    | 10  |        5            | normalization          |
      145                    | 10  |   direct            | UGD03                  |
146 - 155                    | 10  |      100            | UGD03                  |
156 - 158                    | 10  |       20            | UGD03                  |
159 - 161                    | 10  |        5            | UGD03                  |
      170                    | 10  |   direct            | normalization          |
171 - 179                    | 10  |      100            | normalization          |
180 - 182                    | 10  |       20            | normalization          |
183 - 185                    | 10  |        5            | normalization          |
193 - 194                    | 10  |   direct            | EP SUS guide with NiP  |
      195                    | 10  |      100            | EP SUS guide with NiP  |
196 - 198                    | 10  |        5            | EP SUS guide with NiP  |
199 - 201                    | 10  |       20            | EP SUS guide with NiP  |
202 - 241                    | 10  |      100            | EP SUS guide with NiP  |
      249                    | 10  |   direct            | normalization          |
250 - 252                    | 10  |        5            | normalization          |
253 - 255                    | 10  |       20            | normalization          |
256 - 265$^{**}$             | 10  |      100            | normalization          |
273 - 274$^{\dagger\dagger}$ | 10  |   direct            | bad?                   |
275 - 278                    | 10  |       20            | bad?                   |
  1 -  93                    | 11  |       20            | bad?                   |

\* *configuration change, background included* <br/>
$^\dagger$ *to be checked in the data (remove noise at the end)* <br/>
\*\* *... cleaning the area ...* <br/>
$^{\dagger\dagger}$ *(we connect both plexi guides together, as at the beginning)*

## Loading Data

The function used to load the data should take a few strings that determine what types of runs and configuration we want.

In [ ]:
config = "NORM"
run_type = "shot"

# first grab the vector of time bins, which is the same for all our runs
time_bins = np.loadtxt("../data_main/12/sorted/T081217_0022_NORM_s020.tof", usecols = (0))

for filename in os.listdir('../data_main/sorted'):
if ((".tof" in filename) and (config in filename) and (run_type in filename)):

# iterating through the run list
for i in range(0, np.size(run_list)):

    if not (just_monitor_flag):

        # grab the vectors of UCN counts
        data[:,i] = np.loadtxt("../data_main/12/" + str(day_list[i]) + "/T"
        + str(day_list[i]).zfill(2) + "1217_" + str(run_list[i]).zfill(4) + 
        ".tof", usecols = (1))

    if (normalize_flag):

        # normalize the vector according to the start time of the run. Here we
        # are normalizing based on the degradation of the UCN source yield over
        # time
        norm_factor = yield_normalization(run_list[i], day_list[i], 
        yield_fit_parameters)

        data[:,i] * norm_factor

    monitor_data[:,i] = np.loadtxt("../data_monitor/12/" + "T"
    + str(day_list[i]).zfill(2) + "1217_" + str(run_list[i]).zfill(4) + 
    ".tof", usecols = (1))

# get the run times
run_times = get_run_times(run_list, day_list, just_monitor_flag)   


In [5]:
x = 5
y = 3
if ((x == 5) and (y == 3)):
    print("yes")

yes
